In [10]:
import requests
import json
import os
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import pprint

In [11]:
drought_df = pd.read_csv("Resources/dm_export_20000101_20230205 (1).csv")


In [16]:
new_file_name = "attempt2_wildfire.csv"

In [17]:
filepath = os.path.join("Resources", "USDM.json")
with open(filepath) as jsonfile:
    json_data = json.load(jsonfile)
#json_data
url = "https://www.fire.ca.gov/umbraco/api/IncidentApi/List?"
incident_acres_burned = []
incident_latitude = []
incident_longitude = []
incident_start_timestamp = []
incident_county = []
incident_type = []
incident_start_date = []

start_year = 2000
end_year = 2022
counter = start_year

def json_find(year):
    add = f"year={year}&inactive=true"
    response = requests.get(url + add)
    data = response.json()
    return data

def parse_data(year):
    for response in range(len(year)):
        incident = year[response]
        AcresBurned = incident["AcresBurned"]
        latitude = incident["Latitude"]
        longitude = incident["Longitude"]
        timestamp = incident["Started"]
        county = incident["County"]
        types = incident["Type"]
        start_date = incident["StartedDateOnly"]
        if AcresBurned != None:
            if (longitude <= -110) & (longitude >= -130):
                if (latitude <= 43) & (latitude >= 32):
                    incident_acres_burned.append(AcresBurned)
                    incident_latitude.append(latitude)
                    incident_longitude.append(longitude)
                    incident_start_timestamp.append(timestamp)
                    incident_county.append(county)
                    incident_type.append(types)
                    incident_start_date.append(start_date)

while counter <= end_year:
    this_json = json_find(counter)
    parse_data(this_json)
    counter += 1
test = pd.DataFrame(data={'Acres Burned': incident_acres_burned, 'Latitude': incident_latitude, 
                          'Longitude': incident_longitude, 'Incident Start': incident_start_timestamp,
                          'County': incident_county, 'Cause Type': incident_type, "Map Date": incident_start_date,})
test.head()

,Acres Burned,Latitude,Longitude,Incident Start,County,Cause Type,Map Date
0,407.0,36.602575,-118.016510,2013-02-24T08:16:00Z,Inyo,,2013-02-24
1,30.0,34.288877,-116.941311,2013-04-20T17:30:00Z,San Bernardino,,2013-04-20
2,274.0,37.116295,-119.635004,2013-04-30T12:59:00Z,Madera,,2013-04-30
3,6896.0,40.190062,-121.595555,2013-05-01T09:12:00Z,Tehama,,2013-05-01
4,75.0,38.441792,-122.350844,2013-04-30T23:44:00Z,Napa,,2013-04-30


In [18]:
test.to_csv(f"Resources/{new_file_name}")